In [1]:
"""qsvd-v4.py"""

# A = Original matrix
# B = Square root of Matrix A
# U, V, L = Matrices obtained after SVD
# Ut, Vt, Lt = Truncated U, V, L matrices
# Bnp = Matrix obtained from U, V, L with no phase added or truncation done
# Bp = Matrix obtained from U, V, L after truncation and phase added
# Bt = Matrix obtained from U, V, L after truncation, no phase added

""" Import Libraries """
import numpy as np
import scipy.optimize as optimize

### Method 1

In [2]:
""" Optimization Algorithm """
""" New Matrix """
def newMat_2(x, Ut,Lt,Vt):
    l0,l1 = Lt[0], Lt[1]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    for i in range(Ut.shape[0]):
      U_new[i][0] = np.cos(x[i]) / l0
      U_new[i][1] = (np.sin(x[i]) / l1) * np.exp(1j*x[i+Ut.shape[0]])
    return np.dot(np.dot(U_new,np.diag(Lt)), Vt)


def newMat_3(x, Ut,Lt,Vt):
    l0,l1,l2 = Lt[0], Lt[1], Lt[2]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    for i in range(Ut.shape[0]):
      U_new[i][0] = np.cos(x[i]) / l0
      U_new[i][1] = (np.sin(x[i]) / l1) * (np.cos(x[i+Ut.shape[0]])) * np.exp(1j*x[i+2*Ut.shape[0]])
      U_new[i][2] = (np.sin(x[i]) / l2) * (np.sin(x[i+Ut.shape[0]])) * np.exp(1j*x[i+3*Ut.shape[0]])
    return np.dot(np.dot(U_new,np.diag(Lt)), Vt)


def newMat_4(x, Ut,Lt,Vt):
    l0,l1,l2,l3 = Lt[0], Lt[1], Lt[2], Lt[3]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    for i in range(Ut.shape[0]):
      U_new[i][0] = (np.cos(x[i]) / l0) * (np.cos(x[i+Ut.shape[0]]))
      U_new[i][1] = (np.cos(x[i]) / l1) * (np.sin(x[i+Ut.shape[0]])) * np.exp(1j*x[i+3*Ut.shape[0]])
      U_new[i][2] = (np.sin(x[i]) / l2) * (np.cos(x[i+2*Ut.shape[0]])) * np.exp(1j*x[i+4*Ut.shape[0]])
      U_new[i][3] = (np.sin(x[i]) / l3) * (np.sin(x[i+2*Ut.shape[0]])) * np.exp(1j*x[i+5*Ut.shape[0]])
    return np.dot(np.dot(U_new,np.diag(Lt)), Vt)


""" Cost Function """
def costFn(x, Ut,Lt,Vt,B,k):
    if k==2:
      Bp = newMat_2(x, Ut, Lt, Vt)
    elif k==3:
      Bp = newMat_3(x, Ut, Lt, Vt)
    elif k==4:
      Bp = newMat_4(x, Ut, Lt, Vt)
    else:
      return 0
    loss = np.linalg.norm(B**2 - np.abs(Bp)**2)
    return (loss)


""" Calculate Results """
def calcResults(k):
  print("k = ", str(k))
  m,n = 5,5
  print("m = ",m,", n = ",n)
  res = np.zeros((100,2))
  for i in range(100):
      A = np.random.rand(m, n)
      for j in range(m): A[j] /= sum(A[j])
      B = np.sqrt(A)
      U, L, V = np.linalg.svd(B, full_matrices=False)
      initial_guess = np.ones((2*m*(k-1),), dtype=np.longdouble)
      Ut = U[:, :k]
      Vt = V[:k]
      Lt = L[:k]
      Bt = np.dot(np.dot(Ut,np.diag(Lt)), Vt)
      result = optimize.minimize(fun=costFn, x0=initial_guess, args=(Ut,Lt,Vt,B,k),
                                  tol=1e-7, method='Nelder-Mead', options={'maxiter':1e+10})
      res[i][0] = (np.linalg.norm(B**2 - Bt**2))
      res[i][1] = costFn(result.x,Ut,Lt,Vt,B,k)
      if(i%10==0):
        print(i, " ", end='')
  print('\n', res)
  print("\nResult:", res.mean(axis=0))
  return

In [3]:
calcResults(k=2)

k =  2
m =  5 , n =  5
0  10  20  30  40  50  60  70  80  90  
 [[0.26639268 0.19964858]
 [0.32680064 0.26752144]
 [0.25975215 0.19976476]
 [0.26370823 0.22315133]
 [0.28745128 0.22491296]
 [0.34260519 0.30576013]
 [0.23171974 0.20904706]
 [0.15023029 0.12391437]
 [0.35220171 0.30875071]
 [0.30427503 0.28062038]
 [0.31984372 0.25979911]
 [0.23010118 0.21677224]
 [0.2233104  0.13957904]
 [0.38371471 0.36717798]
 [0.27236307 0.22999777]
 [0.47611532 0.45400453]
 [0.32512181 0.29707108]
 [0.29797027 0.29062267]
 [0.29234154 0.28929318]
 [0.34488779 0.28200755]
 [0.26905751 0.18419562]
 [0.25674967 0.23162823]
 [0.3933206  0.29726165]
 [0.28787825 0.27384484]
 [0.37265797 0.36063711]
 [0.31862308 0.28020626]
 [0.31468736 0.27091864]
 [0.21362964 0.19687602]
 [0.30415737 0.18501769]
 [0.28230295 0.26397486]
 [0.19952363 0.1843176 ]
 [0.31643848 0.28800525]
 [0.27508149 0.25822574]
 [0.24863649 0.20643938]
 [0.35353636 0.3334862 ]
 [0.33523102 0.32053436]
 [0.29820767 0.26611913]
 [0.3356900

In [4]:
calcResults(k=3)

k =  3
m =  5 , n =  5
0  10  20  30  40  50  60  70  80  90  
 [[0.1494422  0.04622958]
 [0.14511596 0.11171827]
 [0.15485671 0.05391382]
 [0.1994412  0.09020462]
 [0.08796995 0.02135837]
 [0.14889988 0.09822095]
 [0.17859374 0.12535295]
 [0.14372147 0.07900651]
 [0.09105115 0.04858102]
 [0.2001542  0.12002836]
 [0.3262052  0.21677182]
 [0.19816164 0.17303083]
 [0.14839502 0.10653813]
 [0.14485917 0.08532631]
 [0.13462084 0.09990657]
 [0.19921838 0.14772948]
 [0.18625214 0.12425202]
 [0.11773042 0.02418912]
 [0.06271979 0.04219189]
 [0.17150876 0.10735515]
 [0.13531065 0.11683356]
 [0.2932983  0.07960852]
 [0.23527159 0.14295103]
 [0.25191726 0.17224585]
 [0.21241038 0.15754426]
 [0.1587428  0.10740943]
 [0.14160257 0.11427186]
 [0.1366777  0.08830131]
 [0.06715591 0.05162933]
 [0.17269942 0.17130245]
 [0.15859284 0.13329937]
 [0.18853357 0.17506336]
 [0.14069114 0.11852951]
 [0.13457508 0.09359645]
 [0.10294066 0.05376506]
 [0.14442353 0.11095885]
 [0.09402275 0.0562743 ]
 [0.1432425

### Method 2

In [5]:
""" Optimization Algorithm """
""" New Matrix """
def newMat_2(x, Ut,Lt,Vt):
    l0,l1 = Lt[0], Lt[1]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    U_new[0] = np.cos(x[0]) / l0
    U_new[1] = (np.sin(x[0]) / l1) * np.exp(1j*x[1])
    return U_new


def newMat_3(x, Ut,Lt,Vt):
    l0,l1,l2 = Lt[0], Lt[1], Lt[2]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    U_new[0] = np.cos(x[0]) / l0
    U_new[1] = (np.sin(x[0]) / l1) * (np.cos(x[1])) * np.exp(1j*x[2])
    U_new[2] = (np.sin(x[0]) / l2) * (np.sin(x[1])) * np.exp(1j*x[3])
    return U_new


def newMat_4(x, Ut,Lt,Vt):
    l0,l1,l2,l3 = Lt[0], Lt[1], Lt[2], Lt[3]
    U_new = np.zeros((Ut.shape), dtype=np.cfloat)
    U_new[0] = (np.cos(x[0]) / l0) * (np.cos(x[1]))
    U_new[1] = (np.cos(x[0]) / l1) * (np.sin(x[1])) * np.exp(1j*x[3])
    U_new[2] = (np.sin(x[0]) / l2) * (np.cos(x[2])) * np.exp(1j*x[4])
    U_new[3] = (np.sin(x[0]) / l3) * (np.sin(x[2])) * np.exp(1j*x[5])
    return U_new


""" Cost Function """
def costFn(x, Ut,Lt,Vt,B,k):
    if k==2:
      U_new = newMat_2(x, Ut, Lt, Vt)
    elif k==3:
      U_new = newMat_3(x, Ut, Lt, Vt)
    elif k==4:
      U_new = newMat_4(x, Ut, Lt, Vt)
    else:
      return 0
    Bp = np.dot(np.dot(U_new,np.diag(Lt)), Vt) 
    loss = np.linalg.norm(B**2 - np.abs(Bp)**2)
    return (loss)

""" Calculate Results """
def calcResults(k, a=0, b=0, c=0, d=0):
  print("k = ", str(k))
  df = []
  m,n = 5,5
  print("m = ",m,", n = ",n)
  res = np.zeros((100, 2))
  for i in range(100):
    A = np.random.rand(m, n)
    for j in range(m): A[j] /= sum(A[j])
    B = np.sqrt(A)
    U, L, V = np.linalg.svd(B, full_matrices=False)
    initial_guess = np.ones((2*(k-1),), dtype=np.longdouble)
    Ut = U[:, :k]
    Vt = V[:k]
    Lt = L[:k]
    Bt = np.dot(np.dot(Ut,np.diag(Lt)), Vt)
    U_new = np.zeros(Ut.shape, dtype=np.cfloat)
    for row in range(Ut.shape[0]):
      result = optimize.minimize(fun=costFn, x0=initial_guess, args=(Ut[row, :],Lt,Vt,B[row,:],k),
                                tol=1e-7, method='Nelder-Mead', options={'maxiter':1e+10})
      U_new[row,:] = newMat_2(result.x, Ut[row, :],Lt,Vt)
    
    Bp = np.dot(np.dot(U_new,np.diag(Lt)), Vt) 
    
    res[i][0] = (np.linalg.norm(B**2 - Bt**2))
    res[i][1] = (np.linalg.norm(B**2 - ((Bp)*np.conj(Bp))))
    if i%10==0: print(i, end=' ')
  print('\n', res)
  print("\nResult:", res.mean(axis=0))
  return

In [6]:
calcResults(k=2)

k =  2
m =  5 , n =  5
0 10 20 30 40 50 60 70 80 90 
 [[0.33296574 0.2467738 ]
 [0.21463572 0.20354964]
 [0.27934749 0.2706251 ]
 [0.29066448 0.25842247]
 [0.26145697 0.21980235]
 [0.27235452 0.26466363]
 [0.33020277 0.30011523]
 [0.34981491 0.33045811]
 [0.2931849  0.21186997]
 [0.25521539 0.21538938]
 [0.40322749 0.36163413]
 [0.38838788 0.36473027]
 [0.22502686 0.19988018]
 [0.26939955 0.26600983]
 [0.39337362 0.3592945 ]
 [0.25538534 0.18857402]
 [0.20853851 0.17493168]
 [0.27487369 0.24752736]
 [0.51494708 0.42680938]
 [0.30459913 0.25192497]
 [0.26352954 0.23830546]
 [0.35723782 0.28359639]
 [0.3017545  0.28919355]
 [0.45789718 0.43235574]
 [0.43486455 0.36147293]
 [0.35199966 0.31646097]
 [0.3496127  0.27495315]
 [0.35823394 0.29827724]
 [0.19259833 0.15434384]
 [0.30888049 0.28330452]
 [0.36921626 0.33642274]
 [0.33997364 0.32140586]
 [0.30347132 0.2621814 ]
 [0.35188538 0.29575602]
 [0.26715042 0.25449418]
 [0.26734344 0.2602744 ]
 [0.28089728 0.18499243]
 [0.16629895 0.160870

In [7]:
calcResults(k=3)

k =  3
m =  5 , n =  5
0 10 20 30 40 50 60 70 80 90 
 [[0.03925114 0.64250005]
 [0.0886301  0.41736075]
 [0.09665764 0.44690803]
 [0.16285922 0.32810487]
 [0.24050129 0.92570417]
 [0.15079464 0.55376268]
 [0.1508503  0.62092301]
 [0.19144772 0.85629954]
 [0.1792131  0.40308462]
 [0.35163236 0.78176854]
 [0.08753181 0.21068656]
 [0.14552531 0.61610009]
 [0.20376342 1.04284495]
 [0.1741106  0.91279618]
 [0.34116122 1.10821539]
 [0.19592973 0.54880438]
 [0.08833563 0.31032327]
 [0.22871706 0.68715352]
 [0.17059517 0.45146988]
 [0.11265038 0.73223259]
 [0.25704247 0.64097482]
 [0.20622671 0.45576844]
 [0.20938642 0.61676198]
 [0.19690244 0.99906867]
 [0.11265028 0.46521726]
 [0.13678521 0.71218142]
 [0.19186123 0.60860498]
 [0.25769506 0.53171699]
 [0.16957958 0.6513786 ]
 [0.04506415 0.36262364]
 [0.11506227 0.26790633]
 [0.13097483 0.41128602]
 [0.10415867 0.35269171]
 [0.14193899 0.4171633 ]
 [0.29642213 0.7966635 ]
 [0.18559764 0.98542152]
 [0.10034254 0.7002637 ]
 [0.16672831 0.840865